### This personal notebook describes notes about aws and cli
In this sample we will build in central (frankfurt) region our pred-reclaim api server.

#### Install CLI

```
Windows: https://s3.amazonaws.com/aws-cli/AWSCLI64.msi

Linux or mac requires pip to install, which requires setuptools itself. If you have pip just install the awscli)

SETUPTOOLS:
wget https://pypi.python.org/packages/source/s/setuptools/
setuptools-7.0.tar.gz
tar xvf setuptools-7.0.tar.gz
cd setuptools-7.0
python setup.py install

PIP:
wget https://bootstrap.pypa.io/get-pip.py
python get-pip.py

AWSCLI

pip install awscli

TEST:
aws -version



```
    
    

### Access and security

As we use an existing organisation for now we will not create any users and groups, maybe roles later for you api

```
Configure:

Start configure screen, if you have multiple users using aws cli 
, use different profiles.

root@master:/setuptools-7.0# aws configure (--profile <somename>)
AWS Access Key ID [None]: <secret>
AWS Secret Access Key [None]: <secret>
Default region name [None]: eu-central-1 (or eu-west-1)
Default output format [None]: table

users:
aws iam list-users
aws iam create-user --user-name <user>
aws iam create-login-profile --user-name <user> --password <pwd>
aws iam create-access-key --user <username>

groups:
aws iam create-group --group-name <grp>
aws iam add-user-to-group --user-name <user> --group-name <user>

profiles:
create this file:
{
"Version": "2012-10-17",
"Statement": [
      {
"Effect": "Allow",
"Action": "*",
"Resource": "*"
} ]
}

Grant to user (can also be group):
aws iam put-group-policy --user-name <user> \
--policy-name <all privs for your your org> \
--policy-document file:///tmp/<your file>.json

``` 

### EC2 instances

#### First some general notes

```
ec2 is container running ami images:

two types ami:
Instance store-backed instance via S# (slow)
ebs (elastic block store), shared drive, quick portable, peristant storage, also can handle 16tb instead of 10gb.  but does cost some extra

types:
memory, computer, storage or general optimised, or lastly gpu for graphical/machine learning

pricing:
on-demand , per hour, no cost when not using but higher price
reserved: cheaper on the whole, but you do pay when not using. yu can pay upfront (partly) making it cheaper.
spot: bid on capacity, no guarantee at all.

ebs features:
snapshot stored on s3 for backup and resizing , HA (replication same avail. zone), encryption optional use KMS key service

ebs volumes:
general ssd, providioned iops ssd (quick and expensive, for io intensive apps like db's and hadoop. or magnetic (cheap, backup etc)

``` 

### Install server

``` 
in our keys we will use dil-pred-reclaim-key-pair as name

public key:
aws ec2 create-key-pair --key-name dil-pred-reclaim-key-pair --query 'KeyMaterial' --output text > dil-pred-reclaim-key-pair.pem
 
chmod 400 dil-pred-reclaim-key-pair.pem

security group:
aws ec2 create-security-group --group-name sg_dil-pred-reclaim \
 --description "Security group predictive reclaim api server"

Rules:
aws ec2 authorize-security-group-ingress \
--group-name sg_dil-pred-reclaim \
--protocol tcp --port 22 --cidr 0.0.0.0/0

run (aws linux):
aws ec2 run-instances --image-id ami-b968bad6 \
 --count 1 --instance-type t2.small \
 --security-groups sg_dil-pred-reclaim \
 --key-name dil-pred-reclaim-key-pair \
 --tag-specifications 'ResourceType=instance,Tags=[{Key=Name,Value=dil-pred-reclaim}]'

test :
ssh -i dil-pred-reclaim-key-pair.pem ec2-user@<PUBLIC ip>

how to access an ebs volume you created earlier:
sudo df -h
sudo fdisk -l. (see /dev/<xx> disk not mounted and seen using df
sudo mkfs -t ext4 /dev/<xx>
sudo mkdir /somedir
sudo mount /dev/<xx> /somedir
<edit fstab to make sure it mounts after reboot>

Downside: you cant control ip, unless you create an elastic ip and attach it. Its required if you want to control ip's accessing your server
using security groups
``` 

### EBS volumes

```
use types: gp2, io1, standard (magnetic) :

aws ec2 create-volume \
--size 5 --region eu-central-1 --availability-zone eu-central-1a \
--volume-type gp2

aws ec2 attach-volume \
--volume-id <vol-xxx> \
--instance-id <i-xxx> \
--device /dev/sdg

unmount:
aws ec2 detach-volume \
--volume-id <vol-xxx>

aws ec2 delete-volume \
--volume-id <vol-xxx>

Ebs is replicated, but only in the same avail.zon.
Use snapshots to make backups to other zone.
```

### Virtual Private Cloud (VPC)

```
ITS FREE

In vpc you can create subnets with own policies and routing, seperate from the security groups. Also extra layer: Acl access control list (per subnet) for extra control. connect your own datacentre using virtual private gateway, and connect it using the datacentre gateway.

subnet: range of ip's . two types: public and private subnet (determines internet reachable or not). 

example VPC uses CIDR 10.0.0.0./16 , /16 means 65k addresses, 10.0.0.0 - 10.0.255.255 . Create subnet 10.0.1.0/24 for webserver (public) and 10.0.5.0/24 for databases (private) etc etc (/24 is 256 ip's ).

When creating default vpc: /16 (65k) ip's . az's in default region. a public and private subnet. internet gateway (router between two subnets) . some routetables, security groups and acl's so internet can be used. NOTE: ach instance by default is using the default vpc!

network acl: rules with a number (small is highest prio) allowing or denying access. FInishing with a * to block anything not in a rule.

routing table: diect traffix to from subnet. main route table is default. and it allows traffic inside the vpc.

vpc endpoint: connect vpc to other services (virtual devices, managed by aws)

internet gateway: provide internet connectivity. just add route in routetable of subnet to it and its ready. It also nat's private ip's to public or elastic ip's . the instances themselves dont know the public ip's .

NAT instance: Without internet gatway you can use this nat in public subnet, and route instance in private subnet to this public subnet, which routes to internet (via internet gateway)

DNS Domain name servers. Domain name
```

### VPC creation

### Scenarios

```
scenarios:
1 1 single public subnet . Most simple, all connected to internet
2 2 subnet, 1 publiv 1 private (NAT). Most common. web servers in public, db in private. WIth nat in public for outgoing traffic from private subnet
3 like nr 2 but also vpn access (virtual private gateway). Used to extend on premise datacentre to cloud !!! 
4 only private subnet with vpc . Only internal connections can be made  np internet gateway 

Choice: 3. As we need on premise access, but also expect other clients like klm to connect. But also we need outgoing access as we get lots of internet data (think IOT, flighttracker etc). VPN costs 5 cents per (partial) hour, also for NAT server Extra costs per GB. 
```


### Requirements

```
requirements:

VPC SDH-EU-CENTRAL-PROD-1 192.168.0.0/16  
questions:
- 65k ip's to much? We need no-sql db's, sql-db's , streaming clusters , bigdata cluster. 
- Use central/frankfurt or ireland? Is USA allowed, for example for second Avail. zone?

SDH-EU-CENTRAL-PROD-WEB 192.168.1.0/24 (255 ip's with internet access)
, in zone EU-CENTRAL-1a

SHD-EU-CENTRAL-PROD-DB 192.168.5.0/24 (255 ip's isolated from internet),
in avail. zoine EU-CENTRAL-1b

NAT server in public subnet to have outgoing internet access via internet gateway

How to handle access to S3 etc. For example to ingest flighttracker data to S3 a api in public subnet needs it. So Spark in the private subnet can process it. 
questions:
- full access or custom?

TODO Avail. zones. Boths are in EU-CENTRAL-1a . We need duplicate environment in EU-central-1b? Not sure how this is setup best

Tenancy : dedicated (extra costs but safer)
```

### Manual vpc creation

```
VPC wizard (from vpc dashboard)
- Click option 3 (CANT BE DONE WITHOUT VPN GATEWAY INFO)
So we pick 2. For the nat server we use micro t2, with dil-keypair.
We should create seperate keys first.

Questions:
- Failed with selecting dedicated hardware, but ok when default. Problem?
- Ec2 instance created, but does not seem to be used as NAT server. As 
NAT section shows no NAT server. Meaning private subnet instance cant use internet as NAT is missing. Delete ec2 instance and create NAT instance manually?

```
    


Nice refrence:
https://coderwall.com/p/ndm54w/creating-an-ec2-instance-in-a-vpc-with-the-aws-command-line-interface
    

### Aws cli vpc

```
#!/bin/sh
# Script: Create SDH Aws infrastructure in 1 run
# version 0.1: M.V. Nossin , 27-4-2017
#
# Goal: Create VPC, with public and private subnet. Nat server within the public subnet will provide outgoing internet access for traffic within the private subnet. 
#
# Some pre-notes: The default aws configuration set for the user running
# the script determines the aws region. In my case EU-CENTRAL . Also servers,
# security groups, keys etc will be added in seperate scripts. As these will be reused for all kinds of projects.
#
# TODO: 
#
# - Test VPN actions (requires VPN information)
# - Use two nat servers, one for each Avail. zone ??

#input config
export VPCNAME="SDH-EU-CENTRAL-PROD-1"
export AZ1="eu-central-1a"
export AZ2="eu-central-1b"
export VPCCIDR="192.168.0.0/16"
export SUBNETPUBLICCIDR1="192.168.1.0/24"
export SUBNETPUBLICCIDR2="192.168.5.0/24"
export SUBNETPRIVATECIDR1="192.168.9.0/24"
export SUBNETPRIVATECIDR2="192.168.13.0/24"
export SUBNETPUBLICNAME1="SDH-EU-CENTRAL-PROD-AZ1-PUBLIC"
export SUBNETPUBLICNAME2="SDH-EU-CENTRAL-PROD-AZ2-PUBLIC"
export SUBNETPRIVATENAME1="SDH-EU-CENTRAL-PROD-AZ1-PRIVATE"
export SUBNETPRIVATENAME2="SDH-EU-CENTRAL-PROD-AZ2-PRIVATE"
export ROUTETABLEPUBLICNAME="SHD-EU-CENTRAL-PROD-ROUTETABLE-PUBLIC"
export ROUTETABLEPRIVATENAME="SHD-EU-CENTRAL-PROD-ROUTETABLE-PRIVATE"
export IGWNAME="SDH-EU-CENTRAL-PROD-IGW"
export CUSTOMERGATEWAYIP="12.1.2.3"
export CUSTOMERASN="65534"
export CUSTOMERGWNAME="SDH-EU-CENTRAL-PROD-CUSTOMERGW"
export VPNGWNAME="SDH-EU-CENTRAL-PROD-VPNGW"
export VPNCONNECTIONNAME="SDH-EU-CENTRAL-PROD-VPNCONNECTION"

#Create VPC (dedicated tenancy prevents ec2 server creation!)
echo "Creating vpc"
export VPCID=`aws ec2 create-vpc --cidr-block $VPCCIDR --instance-tenancy default --query 'Vpc.VpcId' --output text`
aws ec2 create-tags --resources $VPCID --tags Key=Name,Value=$VPCNAME
sleep 3

#public subnet
echo "Creating public subnets"
export SUBNETPUBLIC1=`aws ec2 create-subnet --vpc-id $VPCID --cidr-block $SUBNETPUBLICCIDR1 --availability-zone $AZ1 --query 'Subnet.SubnetId' --output text`
aws ec2 create-tags --resources $SUBNETPUBLIC1 --tags Key=Name,Value=$SUBNETPUBLICNAME1
export SUBNETPUBLIC2=`aws ec2 create-subnet --vpc-id $VPCID --cidr-block $SUBNETPUBLICCIDR2 --availability-zone $AZ2 --query 'Subnet.SubnetId' --output text`
aws ec2 create-tags --resources $SUBNETPUBLIC2 --tags Key=Name,Value=$SUBNETPUBLICNAME2
sleep 3

#private subnet
echo "Creating private subnets"
export SUBNETPRIVATE1=`aws ec2 create-subnet --vpc-id $VPCID --cidr-block $SUBNETPRIVATECIDR1 --availability-zone $AZ1 --query 'Subnet.SubnetId' --output text`
aws ec2 create-tags --resources $SUBNETPRIVATE1 --tags Key=Name,Value=$SUBNETPRIVATENAME1
export SUBNETPRIVATE2=`aws ec2 create-subnet --vpc-id $VPCID --cidr-block $SUBNETPRIVATECIDR2 --availability-zone $AZ2 --query 'Subnet.SubnetId' --output text`
aws ec2 create-tags --resources $SUBNETPRIVATE2 --tags Key=Name,Value=$SUBNETPRIVATENAME2
sleep 3

#Internet gateway.
echo "Creating internet gateway"
export IGW=`aws ec2 create-internet-gateway --query 'InternetGateway.InternetGatewayId' --output text`
aws ec2 create-tags --resources $IGW --tags Key=Name,Value=$IGWNAME
aws ec2 attach-internet-gateway --internet-gateway-id $IGW --vpc-id $VPCID
sleep 3

#NAT gateway in public subnet, use for outgoing traffic in private subnet
#Because we cant create new eip (max 5 reached) we will use existing
#non-associated eip
echo "Creating NAT server"
#export EIPNAT1=`aws ec2 allocate-address --domain vpc --query 'Address.AllocationId' --output text`  
export EIPNAT1=`aws ec2 describe-addresses --output text --query 'Addresses[*].[AllocationId,Domain,AssociationId]' | grep vpc | grep None | tail -1 | cut -f 1`
export NATID1=`aws ec2 create-nat-gateway --subnet-id $SUBNETPUBLIC1 --allocation-id $EIPNAT1 --query 'NatGateway.NatGatewayId' --output text`
sleep 3

#TODO: EIPNAT2 FOR SECOND PUBLIC SUBNET IN OTHER AZ
#TODO: Create second NAT for second public subnet

#Route public subnet to internet gateway
echo "Creating route to internet for public subnet"
export ROUTETABLEPUBLIC=`aws ec2 describe-route-tables --output text --query 'RouteTables[*].[RouteTableId,VpcId]' | grep $VPCID | cut -f 1`
aws ec2 create-tags --resources $ROUTETABLEPUBLIC --tags Key=Name,Value=$ROUTETABLEPUBLICNAME
aws ec2 create-route --route-table-id $ROUTETABLEPUBLIC --destination-cidr-block 0.0.0.0/0 --gateway-id $IGW
aws ec2 associate-route-table --route-table-id $ROUTETABLEPUBLIC --subnet-id $SUBNETPUBLIC1
sleep 3

#TODO: Associate EIPNAT2 TO SECOND PUBLIC SUBNET

#Create new main routetable for private subnet , adding route to NAT
echo "Creating rout to NAT server from private subnet
export ROUTETABLEPRIVATE=`aws ec2 create-route-table --vpc-id $VPCID --query 'RouteTable.RouteTableId' --output text`
aws ec2 create-route --route-table-id $ROUTETABLEPRIVATE --destination-cidr-block 0.0.0.0/0 --gateway-id $NATID1
aws ec2 create-tags --resources $ROUTETABLEPRIVATE --tags Key=Name,Value=$ROUTETABLEPRIVATENAME
aws ec2 associate-route-table --route-table-id $ROUTETABLEPRIVATE --subnet-id $SUBNETPRIVATE1
sleep 3

#TODO: Associate route table to second private subnet

#VPN (STEPS NOT TESTED. Current steps depend on no existing customer and vpn gateways, also no vpn connections!)
echo "Creating VPN gateway and connection"
export CUSTOMERGW=`aws ec2 create-customer-gateway --type ipsec.1 --public-ip $CUSTOMERGATEWAYIP --bgp-asn $CUSTOMERASN --query 'CustomerGateway.CustomerGateway' --output text`
export VPNGW=`aws ec2 create-vpn-gateway --type ipsec.1 --query 'VpnGateway.VpnGateway' --output text` #--availability-zone <somestr>
export VPNCONNECTION=`aws ec2 create-vpn-connection --type ipsec.1 --customer-gateway-id $CUSTOMERGW --vpn-gateway-id $VPNGW --query 'VpnConnection.VpnConnection' --output text` #--options false (in case non BGP support)
aws ec2 create-tags --resources $VPNCONNECTION --tags Key=Name,Value=$VPNCONNECTIONNAME
aws ec2 create-tags --resources $CUSTOMERGW --tags Key=Name,Value=$CUSTOMERGWNAME
aws ec2 create-tags --resources $VPNGW --tags Key=Name,Value=$VPNGWNAME
sleep 3

echo "Installation complete"
```

### Aws cli server

### Load balancers
```
Two types:
internet facing, connected to your public subnet.
Or internal to your private. Name for example:
SHD-EU-CENTRAL-PROD-LB-1-PUBLIC (or PRIVATE)
We can use 1 LB per subnet as main entry point providing HA and scalable
acess.

Step 1) create LB. Next add listeners consisting of protocol (http eg), LB external port (eg 80) and internal instance port (eg 80). Good practice is to make these ports different.

Finally associate the subnet, in our case public as we want to use a internet facing LB. If possible use subnet in multiple Avail zones.

Step 2) Assign SG. So instead of per server, we will us it on the LB used to all ec2's servers

step 3) configure security settings
Optional (like HTTPS OR NOT)

step 4) health checks like ping protocol, ping port, path , response time, etc etc to your ec2 instances, and 

Step5) Add ec2 instances

step 6) Add tags

step 7 review and create

``` 


```
example:
Lets say we want to provide an api to expose predictive reclaim without 
using the api platform. We would need the public subnets:

type: application LB: https/https LB in public (as we will use this for an api )

name: SHD-EU-CENTRAL-PROD-LB-1-PUBLIC
scheme: internet-facing
ip address type: ipv4

listener: http , port 80 (as we have non private data like pred-reclaim)

VPC: SDH-EU-CENTRAL-PROD-1

subnet (select both public, as we will use these as pulic api):
SDH-EU-CENTRAL-PROD-AZ1-PUBLIC
SDH-EU-CENTRAL-PROD-AZ2-PUBLIC

security group: (select same SG you would use for a single server):
Let's use our sdh_test server security group: sg_sdh-test

Routing: new target group, name : SHD-EU-CENTRAL-PROD-TG-1-PUBLIC
protocol and port (http and 80 remain unchanged).
Health check on http and / remain default

Add Both ec2 server (1 per avail.zone) to the registered target group.


```


### Initial version Load balancer script

```
#!/bin/sh
# Script: Create Load balancer on SDH Aws infrastructure in 1 run
# version 0.1: M.V. Nossin , 27-4-2017
#
# Goal: Create load balancer on our SDH VPC , in this code we will create a internet facing LB for a public api.
#
# Some pre-notes: The default aws configuration set for the user running
# the script determines the aws region. In my case EU-CENTRAL . Also servers,
#

#input config
export LBNAME="SHD-EU-CENTRAL-PROD-LB-1-PUBLIC"
export SGID="sg-3514af5e"
export SUBNETS="subnet-538f1629 subnet-a69dcece"  #two public subnets, 1 per avail zone
export INSTANCES="Id=i-058871f8ea732470b Id=i-08978824ad560fcdd" #Load balance over these 2 instances
export TGNAME="SHD-EU-CENTRAL-PROD-TG-1-PUBLIC"
export VPCID="vpc-d6c150be"
export PROTOCOL="HTTP"
export PORT="80"
export LISTENER="Protocol="$PROTOCOL",LoadBalancerPort="$PORT",InstanceProtocol="$PROTOCOL",InstancePort="$PORT

#Create target group
echo "Creating target group"
aws elbv2 create-target-group --name $TGNAME --protocol $PROTOCOL --port $PORT --vpc-id $VPCID

#TODO READ ARN RESOURCE PREVIOUS COMMAND

#Register targets in group
echo "Registering targets"
aws elbv2 register-targets --target-group-arn $ARN --targets $INSTANCES 

#create Load balancer (no scheme means internet facing, public):
echo "Creating Load Balancer"
aws elb create-load-balancer --load-balancer-name $LBNAME --listeners $LISTENER --subnets $SUBNETS --security-groups $SGID

echo "Load balancer installation finished"
``` 

By default, all AWS accounts are limited to 5 Elastic IP addresses per region, because public (IPv4) Internet addresses are a scarce public resource. We strongly encourage you to use an Elastic IP address primarily for the ability to remap the address to another instance in the case of instance failure, and to use DNS hostnames for all other inter-node communication.


Test:
aws ec2 run-instances --image-id ami-9abea4fb --count 1 --instance-type t2.micro --key-name <key*> --security-group-ids <sg-*> --subnet-id <subnet*> --associate-public-ip-address

## scaling groups

Lets skip this for now. 

#### RDS relation. database

```
USe mysql,oracle,sqlserver,postgress or mariadb as a servivce

HA solution: by default multiple az are used already. Use a read only db in other regions to make sure the db's can be read even if region fails.

manual install:

select rds
select the two private subnets are subnet groups
select engine (eg mysql)
in case of production select this to make sufe mult az is used
db details (license etc)
last advanced (vpc/subnet but also backup recovery

To conne t from you public server:
sudo yum install mysql
mysql -u admin -h <endpoint, dns for db> -p
show databases;

CREATE TABLE doge
(
idint(11) NOT NULL auto_increment,
namevarchar(255),
description text,
sizeenum('small','medium','large'),
date timestamp(6),
  PRIMARY KEY (id)
);

INSERT INTO doge (name,description ,size,date) VALUES('Xena','Black
Labrador Retreiver','medium',NOW());
INSERT INTO doge (name,description ,size,date) VALUES('Betsy','Browndachs
hund','medium',NOW());
INSERT INTO doge (name,description ,size,date) VALUES('Shark','Mix bread-
Half dachshund','small',NOW());

creating read only db's and promoting db in case of failure is also possible.

```

### tips
```
To begin with, always monitor your DB instances for overall performance and usage. Leverage CloudWatch and its metrics to set thresholds and customized alarms that notify you in case of any issues.
• Additionally, you can also enable event notifications for your DB instance that will inform you of all the events that occur with your instance.
• Leverage Multi-AZ deployments in conjunction with Read Replicas to increase the overall availability of your database along with its performance.
• Always enable automatic snapshots for your DB instances. Also take manual snapshots of your DB instances before performing any maintenance activities on them.
• For a small to medium-sized database, set the storage type of your DB instance to General Purpose (SSD).
• If your database has a high performance requirement, then do use the DB instances with Provisioned IOPS.
• Tune your options group as well as your parameters group to improve your database's overall performance.
• Secure your DB instances with a proper security group and encryption facilities. Also remember to assign and use IAM users with specific rights and privileges.
```

### Dynamo DB

Already create a manual table. Lets not use CLI, as we will also use this
table using code, lets do all steps in python. The creation script
can also be execute as part of the infra code.

In [ ]:
#!/Users/michelnossin/anaconda/bin/python

#We will use $HOME/.aws/credentials and config set by aws cli. (and not boto.client())
#python -m pip install boto3
import boto3

# Get the service resource.
dynamodb = boto3.resource('dynamodb')

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName='flight_events',
    KeySchema=[
        {
            'AttributeName': 'day',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'flight',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'day',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'flight',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='flight_events')

# Print out some data about the table.
print(table.item_count)



In [ ]:
#!/Users/michelnossin/anaconda/bin/python

#We will use $HOME/.aws/credentials and config set by aws cli. (and not boto.client())
#python -m pip install boto3
import boto3

# Get the service resource.
dynamodb = boto3.resource('dynamodb')

table = dynamodb.Table('flight_events')

table.put_item( Item= { "day" : "2-may-2107" , "time" : "14:14" , "flight" : "KL1234" , "flt" : { "cdm" : { "eibt" : "2017-05-03T22:39:00.000+02:00" }, "flight" : { "flightArrival" : { "blt" : [ { "belt" : "16A" } ] }, "fltnr" : "CND312" }, "prkStand" : { "prkStatus" : [ { "prk" : [ { "gateType" : "G", "prkValue" : "Z02" }, { "gateType" : "R", "prkValue" : "D02" } ] } ] }, "sdatetime" : "2017-05-03T22:50:00.000+02:00", "arrdep" : "A" } } )

#learning: exact primary and sort key have to be in main level. So as our providers does not give it in this form
#we will add these ourself


### S3 . Simple Storage Service

```
3 cents per gb or less (standard is HA, all the way to RRS Reduced Redundancy.
Its not a file system. Each directory and file gets an url. All elements are stored flat and therefore scales well.
Using S3 manually can be done easily from the console.

Use S3cmd as hdfs alternative (put and get objects):
wget http://sourceforge.net/projects/s3tools/files/s3cmd/1.6.0/s3cmd-
1.6.0.tar.gz
tar -xvfs3cmd-1.6.0.tar.gz
cd s3cmd-1.6.0
python setup.py install
s3cmd --configure

list buckets: s3cmdlss3://<BUCKET>
s3cmd put -r /opt s3://<BUCKET_NAME>

put logfiles:
BUCKET=httpd-logger
INSTANCE_ID=`curl --silent http://169.254.169.254/latest/meta-data/
instance-id`
  /usr/bin/s3cmd -m text/plain sync /etc/httpd/logs/access_log*
s3://${BUCKET}/httpd/access_log/instance=${INSTANCE_ID}/
  /usr/bin/s3cmd -m text/plain sync /etc/httpd/logs/error_log*
s3://${BUCKET}/httppd/error_log/instance=${INSTANCE_ID}/


```





```
Using the CLI:
aws s3 <Command> [<Arg> ...]

eg 
aws s3 cp /tmp/foo/ s3://bucket/ --recursive \
    --exclude "*" --include "*.jpg" --include "*.txt"
    

Lets try:
aws s3 mb s3://michel-pred-reclaim-test
echo Michel is the best > /tmp/michel.txt
aws s3 mv /tmp/michel.txt s3://michel-pred-reclaim-test/michel.txt

```

### Route 53 DNS

```

Create public hosted zone in case of website to public
. You will see 4 created dns servers, and the SOA which is the Start of Autohority describing these servers.

click create recordset:
You create Cname for create canonical names mapping names to another
, like a record containing an ip or an AAAA record with am ipv6 record. etc. In value , copy paste for example the DNS for the load balancer. In Name just give the name for you domain (example testlb)
use simple routing for single resource, weighted routing. if you have 5 webservers and 1 servers gets weight 1 , rest 2, the others will be twice as busy. latency routing checks region users and forwards to best region with least latency. geolocation, based on location, failover routing is used if  a region is down. Lets use latency. In set id use a name
with the region (EU-CENTRAL-1-DC for example , DC stands for data centre.


Create visual route flow, which is way more work.


TODO: Describe AWAS CLI

```

TODO: LAMBDA OR CONTAINER SERVICES FOR DOCKER